## 予測モデルの作り方・評価の仕方
以下の流れを、簡単なモデルを例にやってみる。なにか助けになれば幸いです。

1. 何らかのアプローチでモデルを作る
1. 内部評価をする
1. 提出をする

## 1. モデルを作る
今回はシンプルに、trainの死亡フライト数の分布を使った簡単な予測モデルを作ってみる。

今回の思想
* 死亡するフライト数を予測して、最後のフライト数から引き算すれば残存寿命になるのはず（残存寿命 = 死亡フライト数 - そのときのフライト数）
* どのエンジンもだいたい同じフライト数で死ぬでしょみたいなニュアンス

### データの読み込み

In [47]:
import sys
sys.path.append('D://isid_phm/src/')

In [48]:
from libs.Dataset import Dataset

In [49]:
df = Dataset().load_data(reproduce=True, cutoff=False)

In [50]:
# train, valid, testのデータに分割
train = df[(df['is_train'] == 1) & (df['is_valid'] == 0)]
valid = df[df['is_valid'] == 1]
test = df[df['is_train'] == 0]

### 予測モデルをつくる

各エンジンのtrainの死亡フライト数の平均を、testの死亡フライト数の予測値とする

In [51]:
# 各エンジン別に死亡フライト数を集計
dead_flight = train.groupby('engine_no')['dead_duration'].first()

In [52]:
dead_flight.head()

engine_no
Train_Engine_001    148
Train_Engine_002    268
Train_Engine_003    205
Train_Engine_004    234
Train_Engine_005    153
Name: dead_duration, dtype: int64

In [53]:
dead_mean = dead_flight.mean()
dead_mean

206.12173913043478

この値を予測死亡フライト数とする

## 2. 内部評価をする

さっきの予測がどれくらい正しいのかを、validデータを使って評価する。
* 一日１submitしか出来ないので、内部検証が出来ないとモデルの改善等のPDCAスピードが遅くなるため
* validは、元データのtrainデータのエンジンNo1～260の中からランダムに30個エンジンくらいを選んだもの
* validもtrainと同様、正解の死亡フライト数を持っている
* trainにvalidは含まれない(含めてしまうと、testと同じような状況での評価ができない）

In [54]:
valid['engine_no'].nunique()

30

In [55]:
train['engine_no'].nunique()

230

このように、validは30機分のエンジンを評価用に残してある

In [56]:
import pandas as pd

# 各エンジンに対する死亡フライト数の予測値
predict_dead_valid = pd.Series([dead_mean]*(valid['engine_no'].nunique()))
predict_dead_valid.head()

0    206.121739
1    206.121739
2    206.121739
3    206.121739
4    206.121739
dtype: float64

In [57]:
# 本当のvalidの死亡フライト数
true_dead_valid = valid.groupby(['engine_no'])['dead_duration'].first()
true_dead_valid.head()

engine_no
Train_Engine_008_reuse_0    150
Train_Engine_009_reuse_0    198
Train_Engine_038_reuse_0    176
Train_Engine_046_reuse_0    200
Train_Engine_056_reuse_0    162
Name: dead_duration, dtype: int64

ここで本当の死亡フライト数と予測したフライト数を平均絶対誤差で評価する
* 残存寿命で評価しない理由は、「残存寿命 = 死亡フライト数 - 現在のフライト数」なので、残存寿命で評価しても死亡フライト数でも同じ値になるため
* 平均絶対誤差は提出スコアも同じ評価基準で評価されるため、同じものを用いる

In [58]:
from sklearn.metrics import mean_absolute_error

mean_absolute_error(true_dead_valid, predict_dead_valid)

34.06579710144927

内部評価ができた!

* この値がいいかどうかは、他の手法/アプローチと比較することで判断する
* この値が公式のsubmitスコアとほぼ同じになれば理想。そうすればsubmitすることなく、内部評価を繰り返せば検討が可能になる。
* 同じにならなくても、この内部評価スコアとsubmitスコアに相関が出れば、検討には十分使える

## 3. 提出をする

いい感じのモデルができたら、そのモデルを使ってtestデータの予測を行い、提出の形に整える

### testデータの予測

In [59]:
# testの予測死亡フライト数
predict_dead_test = pd.Series([dead_mean]*(test['engine_no'].nunique()))
predict_dead_test.head()

0    206.121739
1    206.121739
2    206.121739
3    206.121739
4    206.121739
dtype: float64

今回はシンプルなモデルのため上記の形のなっているが、回帰分析などのモデルでは
下記のような作ったモデルに対して予測を行う
```python
predict_dead_test = MySuperModel.predict(test)
```

ここで、提出したいのは死亡フライト数ではなく、残存寿命なのでその計算を行う

In [60]:
# 各エンジンのtestの最後に記録されていたフライト数
last_flight_test = test.groupby('engine_no')['duration'].last()
last_flight_test.head()

engine_no
Test_Engine_001     57
Test_Engine_002    195
Test_Engine_003    124
Test_Engine_004     49
Test_Engine_005    162
Name: duration, dtype: int64

In [61]:
# 残存寿命 = 予測死亡フライト数 - 記録されている最後のフライト数
predict_dead_test.index = last_flight_test.index
predict_rul = predict_dead_test - last_flight_test
predict_rul.head()

engine_no
Test_Engine_001    149.121739
Test_Engine_002     11.121739
Test_Engine_003     82.121739
Test_Engine_004    157.121739
Test_Engine_005     44.121739
dtype: float64

残存寿命の予測値ができたので、これをsubmitの形にして提出！